In [1]:
import pandas as pd

ema_frame = pd.read_excel("../ema_ingest/ema_list.xlsx")
pmda_frame = pd.read_excel("../pmda_ingest/pmda_list.xlsx")
fda_frame = pd.read_excel("../orange_book_ingest/fda_ob_list.xlsx")

In [44]:
def merge_columns (name1, name2, df, newname):
    df[newname]=df.apply(lambda x:'%s|%s' % (x[name1],x[name2]),axis=1)
    df.drop(name1,axis=1,inplace=True)
    df.drop(name2, axis=1,inplace=True)

    for idx, row in df.iterrows():
        if 'nan' in row[newname]:
            df[newname][idx] = row[newname].replace('nan', "").replace('|','')
    return df

def merge_identical_subcolumns(colname, df, delimiter):
    for idx, row in df.iterrows():
        subColumns = row[colname].split(delimiter)
        if len(subColumns)==2:
            if subColumns[0] == subColumns[1]:
                df[colname][idx] = subColumns[0]

    return df

df1 = pd.merge(ema_frame, pmda_frame, on="single_ID", how="outer")
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]
df1 = merge_columns('Therapy_Name_x', 'Therapy_Name_y', df1, 'Therapy_Name')
df1 = merge_columns('ID_Label_x', 'ID_Label_y', df1, 'ID_Label')
df1 = merge_columns('Combination_Therapy_x', 'Combination_Therapy_y', df1, 'Combination_Therapy')
df1 = merge_columns('Ingredient_IDs_x', 'Ingredient_IDs_y', df1, 'Ingredient_IDs')
df1 = merge_identical_subcolumns('Combination_Therapy', df1, "|")
df1 = merge_identical_subcolumns('Therapy_Name', df1, "|")
df1 = merge_identical_subcolumns('ID_Label', df1, "|")
df1.to_excel("EMA_PMDA_MERGE.xlsx")


df2 = pd.merge(df1, fda_frame, on="single_ID", how="outer")
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
df2 = merge_columns('ID_Label_x', 'ID_Label_y', df2, 'ID_Label')
df2 = merge_columns('Therapy_Name_x', 'Therapy_Name_y', df2, 'Therapy_Name')
df2 = merge_columns('Combination_Therapy_x', 'Combination_Therapy_y', df2, 'Combination_Therapy')
df2 = merge_columns('Ingredient_IDs_x', 'Ingredient_IDs_y', df2, 'Ingredient_IDs')

for idx, row in df2.iterrows():
    if not row['Approved_Europe']== True:
        df2['Approved_Europe'][idx] = False
    if not row['Approved_Japan']== True:
        df2['Approved_Japan'][idx] = False
    if not row['Approved_USA']== True:
        df2['Approved_USA'][idx] = False

df2 = merge_identical_subcolumns('Combination_Therapy', df2, "|")
df2 = merge_identical_subcolumns('Therapy_Name', df2, "|")
df2 = merge_identical_subcolumns('ID_Label', df2, "|")

df2.to_excel("PMDA_EMA_FDA_MERGE.xlsx")

df2.to_csv("drugList.tsv", sep='\t')



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/var/folders/gm/d722cfgx1tdd8x4b5b1kd6j80000gn/T/ipykernel_72943/133755692.py:45: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in